In [1]:
import httpx
import pandas as pd
import logging
import calendar
import unidecode
import openpyxl
import locale
from datetime import datetime, timedelta
import os

In [ ]:
# Definir a localização para português
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

In [2]:
# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%d/%m/%Y %H:%M:%S'
)

logger = logging.getLogger(__name__)

# Diminuir o nível de log para o httpx e outros loggers de terceiros
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [ ]:
# Diminuir o nível de log para o httpx e outros loggers de terceiros
logging.getLogger("httpx").setLevel(logging.WARNING)

ids_nao_mapeados = {
    "Santana Do Livramento": 430016,
    "São Félix Do Araguaia": 5107859,
    "São Luiz Do Paraitinga": 3550001,
    "Mal. Cândido Rondon": 4114609 , 
    "Mal. Candido Rondon":4114609
}
Estação = "Automatica"

def limpar_nome_municipio(nome_municipio: str) -> str:
    """
    Remove partes do nome do município que não são relevantes para a busca do código IBGE.
    """
    # Remove tudo após o hífen ou parêntese
    nome_limpo = nome_municipio.split("- ")[0].split(" (")[0].strip()
    return nome_limpo

def fazer_requisicao(url: str, timeout: int = 10) -> list:
    """
    Faz a requisição para a API e retorna os dados.
    
    Parâmetros:
        url (str): URL da requisição.
        timeout (int): Tempo máximo de espera para a resposta.

    Retorna:
        list: Dados da resposta em formato JSON ou uma lista vazia.
    """
    try:
        response = httpx.get(url, timeout=timeout)
        response.raise_for_status()  # Levanta exceção para códigos HTTP >= 400
        dados = response.json()

        for item in dados:
            item["Estação"] = "Automatica"  

        return dados
    except httpx.HTTPStatusError as e:
        logger.error(f"Erro HTTP {e.response.status_code} ao fazer requisição para {url}")
        return []
    except httpx.RequestError as e:
        logger.error(f"Erro na requisição: {e}")
        return []


def obter_codigo_ibge(nome_municipio: str) -> int:
    # Limpa o nome do município
    nome_municipio_limpo = limpar_nome_municipio(nome_municipio)
   

    # Verifica se o município está no dicionário de mapeamento manual
    if nome_municipio_limpo in ids_nao_mapeados:
        return ids_nao_mapeados[nome_municipio_limpo]  # Retorna o código diretamente do dicionário

    # Se o município não estiver no dicionário, vamos tentar a API do IBGE
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    municipios = fazer_requisicao(url)

    if not municipios:  # Caso a API não retorne dados
        logger.error(f"Erro ao obter dados do IBGE. Informe manualmente o código para {nome_municipio_limpo}.")
        return solicitar_codigo_manual(nome_municipio_limpo)

    # Se o município for encontrado na API
    for municipio in municipios:
        if municipio['nome'].lower() == nome_municipio_limpo.lower():
            return municipio['id']

    # Se o município não foi encontrado na API, solicita o código manualmente
    logger.warning(f"Código IBGE para {nome_municipio_limpo} não encontrado. Será necessário inserir manualmente.")
    return solicitar_codigo_manual(nome_municipio_limpo)

def solicitar_codigo_manual(nome_municipio: str) -> int:
    while True:
        try:
            codigo_manual = int(input(f"Informe o código IBGE para {nome_municipio}: "))
            ids_nao_mapeados[nome_municipio] = codigo_manual  # Salva para futuras consultas
            return codigo_manual
        except ValueError:
            print("Código inválido. Insira um número válido.")

def formatar_data_brasileira(data_iso: str) -> str:
    """
    Converte a data do formato ISO para o formato brasileiro.

    Parâmetros:
        data_iso (str): Data em formato ISO (YYYY-MM-DD).

    Retorna:
        str: Data no formato DD/MM/YYYY.
    """
    try:
        data_formatada = datetime.strptime(data_iso, '%Y-%m-%d').strftime('%d/%m/%Y')
        return data_formatada
    except ValueError:
        logger.error(f"Erro ao formatar a data: {data_iso}")
        return "Data inválida"

def formatar_temperatura(temperatura: str) -> float:
    """
    Converte a temperatura para um número float.

    Parâmetros:
        temperatura (str): Valor da temperatura.

    Retorna:
        float: Temperatura formatada.
    """
    try:
        return round(float(temperatura), 1)  # Garante que seja float com 1 casa decimal
    except (TypeError, ValueError):
        logger.error(f"Erro ao formatar a temperatura: {temperatura}")
        return None  # Ou pode retornar float('nan') se preferir um valor numérico inválido

def calcular_intensidade(temp_min: str) -> str:
    """
    Calcula a intensidade da geada com base na temperatura mínima.

    Parâmetros:
        temp_min (str): Temperatura mínima.

    Retorna:
        str: Intensidade da geada.
    """
    try:
        temp_min_float = float(temp_min)
        if temp_min_float < 1:
            return "Forte"
        elif temp_min_float <= 3:
            return "Moderada"
        else:
            return "Fraca"
    except (TypeError, ValueError):
        return "N/A"

def buscar_id_por_nome(cidades: list, nome_cidade: str) -> int:
    """
    Busca o ID do município pelo nome.

    Parâmetros:
        cidades (list): Lista de dicionários com os dados dos municípios.
        nome_cidade (str): Nome do município a ser buscado.

    Retorna:
        int: ID do município ou -1 se não for encontrado.
    """
    nome_cidade_normalizado = unidecode.unidecode(nome_cidade).lower()
    for cidade in cidades:
        nome_cidade_api_normalizado = unidecode.unidecode(cidade["nome"]).lower()
        if nome_cidade_api_normalizado == nome_cidade_normalizado:
            return cidade["id"]
    return -1

def extrair_dados_geada(folder_path: str = r"C:\Users\ana.brum\Desktop\DadosMeteorologicos"):
    """
    Extrai os dados de geadas da API e salva em um arquivo Excel.
    
    Parâmetros:
        folder_path (str): Caminho da pasta onde o arquivo Excel será salvo.
    """
    cidades = fazer_requisicao("https://servicodados.ibge.gov.br/api/v1/localidades/municipios")
    if not cidades:
        logger.error("Não foi possível obter a lista de municípios.")
        return

    data_inicio = datetime(2017, 1, 1)
    data_fim = datetime(2024, 9, 30)
    dados_tratados = []

    while data_inicio <= data_fim:
        ano = data_inicio.year
        mes = data_inicio.month
        nome_mes = data_inicio.strftime('%B').capitalize()

        logger.info(f"Extraindo dados para: {nome_mes} de {ano}")

        primeiro_dia = data_inicio.strftime("%Y-%m-%d")
        ultimo_dia = (data_inicio + timedelta(days=calendar.monthrange(ano, mes)[1] - 1)).strftime("%Y-%m-%d")

        url = f"https://apitempo.inmet.gov.br/geada/{primeiro_dia}/{ultimo_dia}/AUTOMATICA"
        dados = fazer_requisicao(url)

        # Tratamento dos dados
        if dados:
            for item in dados:
                uf = item.get("UF", "N/A")
                nome_cidade = item.get("NOME", "N/A").title()
                nome_cidade_limpo = limpar_nome_municipio(nome_cidade)  # Limpa o nome do município
                data_ocorrencia = formatar_data_brasileira(item.get("DT_MEDICAO"))
                temp_min = item.get("TEMP_MIN")
                temperatura_formatada = formatar_temperatura(temp_min)
                intensidade = calcular_intensidade(temp_min)
                id_cidade = buscar_id_por_nome(cidades, nome_cidade_limpo)
                item["Estação"] = "Automatica"
    

                # Adicionando os dados tratados à lista
                dados_tratados.append([id_cidade, uf, nome_cidade_limpo, data_ocorrencia, temperatura_formatada, intensidade,Estação])

        # Avançar para o próximo mês
        data_inicio += timedelta(days=calendar.monthrange(ano, mes)[1])

    # Criar DataFrame com os dados tratados
    colunas = ["Cod. IBGE", "Uf", "Município", "Dia de ocorrência", "Temperatura Mínima", "Intensidade", "Estação"]
    df = pd.DataFrame(dados_tratados, columns=colunas)

    # Certificando-se de que a pasta existe
    os.makedirs(folder_path, exist_ok=True)

    # Caminho completo do arquivo Excel
    excel_file = os.path.join(folder_path, "dados_geada_automatico.xlsx")
    df.to_excel(excel_file, index=False)
    logger.info(f"Dados extraídos e salvos com sucesso no arquivo '{excel_file}'.")

if __name__ == "__main__":
    extrair_dados_geada()

17/02/2025 00:59:15 - INFO - Extraindo dados para: January de 2017
17/02/2025 00:59:16 - INFO - Extraindo dados para: February de 2017
17/02/2025 00:59:16 - INFO - Extraindo dados para: March de 2017
17/02/2025 00:59:17 - INFO - Extraindo dados para: April de 2017
17/02/2025 00:59:17 - INFO - Extraindo dados para: May de 2017
17/02/2025 00:59:18 - INFO - Extraindo dados para: June de 2017
17/02/2025 00:59:19 - INFO - Extraindo dados para: July de 2017
17/02/2025 00:59:22 - INFO - Extraindo dados para: August de 2017
17/02/2025 00:59:23 - INFO - Extraindo dados para: September de 2017
17/02/2025 00:59:23 - INFO - Extraindo dados para: October de 2017
17/02/2025 00:59:24 - INFO - Extraindo dados para: November de 2017
17/02/2025 00:59:25 - INFO - Extraindo dados para: December de 2017
17/02/2025 00:59:25 - INFO - Extraindo dados para: January de 2018
17/02/2025 00:59:26 - INFO - Extraindo dados para: February de 2018
17/02/2025 00:59:26 - INFO - Extraindo dados para: March de 2018
17/02/